In [1]:
B_SIZE = 16

In [2]:
import json

In [3]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model = model.to(device = 'cuda')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def paraphrase(sents):

  text =  ["summarize: " + sent + " </s>" for sent in sents]

  encoding = tokenizer.batch_encode_plus(text,pad_to_max_length=True, return_tensors="pt")

  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

  outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      max_length=256,
      do_sample=True,
      top_k=500,
      top_p=1.25,
      early_stopping=True,
      num_return_sequences=5
  )

  para = []

  for sent_i in range(len(sents)):
    for i in range(5):
      cur_idx = 5*sent_i + i
      line = tokenizer.decode(outputs[cur_idx], skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if(line != sents[sent_i]):
        para.append(line)
        break
      if(i == 4):
        para.append(line)
      
  return para

In [6]:

f = open('valid.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
data_para = []
for b_num in tqdm(range(int(len(data)/B_SIZE))):
    batch = data[b_num*B_SIZE:b_num*B_SIZE + B_SIZE]
    sents = [i['sent'] for i in batch ]
    para = paraphrase(sents)
    for i in range(len(sents)):
        data_para.append(batch[i].copy())
        batch[i]['sent'] = para[i]
        data_para.append(batch[i])

data_para

with open('valid_para.json', 'w') as file:
    json.dump(data_para, file)

100%|███████████████████████████████████████| 1253/1253 [07:45<00:00,  2.69it/s]
